# Cogo Data Science technical interview task

## What will the format be?

We have prepared a coding task that we would like you to prepare for the interview. You will find the data and the data dictionary attached. The data consists of 2070 rows of Cogo app users (1 row per user), their demographic data as well as their sustainability values (badges) and the climate actions they committed to in the app (actions). If a person has selected a sustainability badge, or committed to a climate action, it is marked as 1 , otherwise 0.

We don’t expect you to spend more than 2 hours on the task!

This is an exploratory exercise and we want you to have fun with the data. We’d like you to work in Python, maybe Jupyter Notebook so you can take us through the code and results on the day of the interview.

### Here are a couple of suggestions you could be looking at:
- Exploratory Analysis: Spend some time exploring the data, using some data
visualisation and descriptive statistics. Tell us about interesting things you come
across.
- Clustering: Can you identify any clusters in the Badge data, or Action data? Can you
infer one from the other? If you find any clusters in the data, how would you interpret
those?
- Application: Based on your findings, what are some recommendations you would
take to the product team?

In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from lib import DATA_DIRECTORY, USER_DATA_RAW_PATH, DATA_DICTIONARY_PATH, USER_DATA_PATH

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read the provided data files

And take a look.

In [5]:
user_data_raw = pd.read_csv(USER_DATA_RAW_PATH)
data_dictionary = pd.read_csv(DATA_DICTIONARY_PATH)

# I like to keep a copy of the raw data handy in case I mess up a column
# we won't modify user_data_raw, only user_data
user_data = user_data_raw.copy()

In [6]:
user_data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2070 entries, 0 to 2069
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   person_id                      2070 non-null   object
 1   gender                         2070 non-null   object
 2   age                            2070 non-null   object
 3   action_no_red_meat             2070 non-null   int64 
 4   action_vegan                   2070 non-null   int64 
 5   action_vegetarian              2070 non-null   int64 
 6   action_composting              2070 non-null   int64 
 7   action_using_renewable_energy  2070 non-null   int64 
 8   action_buying_secondhand       2070 non-null   int64 
 9   action_meat_free_monday        2070 non-null   int64 
 10  action_use_public_transport    2070 non-null   int64 
 11  action_drive_electric_vehicle  2070 non-null   int64 
 12  badge_reduce_waste             2070 non-null   int64 
 13  bad

No nulls!

## Let's investigate these columns a little more closely

## person_id

The data dictionary says this is unique so I'd like to know that there are no duplicates in this column.

In [7]:
duplicated_person_id = (
    user_data_raw
    .person_id
    .loc[user_data_raw
         .person_id
         .duplicated(keep = 'first')])
print(f'There are {duplicated_person_id.shape[0]} duplicated person_ids.')

There are 0 duplicated person_ids.


That is good news.

## gender

Take a look at the values and their counts.

In [8]:
# `to_frame()` here is just because I like the look of the output more
user_data_raw.gender.value_counts().sort_index().to_frame()

,gender
Female,1017
Male,606
Other/Unknown,447


Much larger proportion of women than men. I do recall reading that many men avoid pro-environmental activities because they are seen as being "feminine".

## age

Take a look at the values and their counts.

In [9]:
100 * user_data_raw.age.value_counts(dropna = False, normalize = True).sort_index().to_frame()

,age
age unknown,19.613527
between 31 and 40,16.135266
between 41 and 55,10.772947
over 55,1.111111
under 30,52.367150


The first gotcha is that people who are 30 years old aren't included in this table.

Intervals are `"under 30"` and `"between 31 and 40"` so people of the age of 30 are excluded (perhaps included in `"age unknown"`?)

The second gotcha is that these ages *should* have an inherent order but in the current encoding they don't because they begin with alphabetical characters. We'll fix that because when it comes to plot against these ages I'd like the plotting functions to be able to interpret their order in a sensible way.

In [10]:
# turn the age column into an Series of intervals which pandas will interpret as ordinal
# on a larger dataset I might use regex to match the patterns here and make the Interval objects automatically
# for this task I'm just going to map from one to another
# here we are assuming that the youngest possible user is 18 and the oldest is 100

age_interval_map = {
    'age_unknown'       : pd.NA,                                # sadly there is no such thing as a null Interval
    'between 31 and 40' : pd.Interval(31, 40, closed = 'both'), # assuming this range is inclusive based on the ranges either side
    'between 41 and 55' : pd.Interval(41, 55, closed = 'both'), # following the convention from the [31, 40] Interval
    'over 55'           : pd.Interval(56, 100, closed = 'both'),# this is ambiguous, but the age of 55 already exists in the [41, 55] Interval
    'under 30'          : pd.Interval(18, 29, closed = 'both')  # this isn't ambiguous. 'under 30' means these people have ages <=29
}

# map the age to the interval
age_intervals = (
    user_data_raw
    .age
    .map(age_interval_map))

# let's compare
pd.concat([user_data_raw.age, age_intervals], axis = 1)

,age,age
0,under 30,"[18.0, 29.0]"
1,between 31 and 40,"[31.0, 40.0]"
2,between 31 and 40,"[31.0, 40.0]"
3,under 30,"[18.0, 29.0]"
4,between 31 and 40,"[31.0, 40.0]"
...,...,...
2065,age unknown,NaN
2066,under 30,"[18.0, 29.0]"
2067,under 30,"[18.0, 29.0]"
2068,age unknown,NaN


In [11]:
# looks good. Update the ages in user_data and try again with an ordered value count
user_data['age'] = age_intervals

100 * user_data.age.value_counts(dropna = False, normalize = True).sort_index().to_frame()

,age
"[18.0, 29.0]",52.367150
"[31.0, 40.0]",16.135266
"[41.0, 55.0]",10.772947
"[56.0, 100.0]",1.111111
NaN,19.613527


Now the ages are ordered in a natural way, and the unknown value is a data type which pandas knows how to handle.

We see that slightly over half of our sample is younger than 30, while users over the age of 55 are very much in the minority (1.1%).

We lack data for roughly 20% of the sample.

## actions

There's a few actions so we'll speed this up by isolating just the column names that start with `action`.

These columns represent what actions each user has committed to performing in order to lower their carbon footprints.

In [12]:
# this could probably be more pandasy, but here a list comprehension will accomplish this just fine
action_names = [n for n in user_data_raw.columns if n.startswith('action')]
actions = user_data_raw[action_names]
actions

,action_no_red_meat,action_vegan,action_vegetarian,action_composting,action_using_renewable_energy,action_buying_secondhand,action_meat_free_monday,action_use_public_transport,action_drive_electric_vehicle
0,0,0,0,0,0,1,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,1,0,1,1,0,0,0,0
3,0,1,0,1,1,1,0,0,0
4,1,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...
2065,0,1,0,0,0,1,0,0,0
2066,0,0,0,0,0,1,0,0,0
2067,0,0,1,0,0,0,0,0,0
2068,0,1,0,1,1,1,0,0,0


We can summarise the content of the actions selections by applying a `value_count()` method to all the columns.

This will tell us the percentage of our sample who have signed up to a specific action. It will also indicate if we have any anomalous values (like a `2`) to be concerned about.

In [13]:
100 * (
    actions
    .apply(lambda x : x.value_counts(dropna = False, normalize = True))
    .fillna(0.)
    .T
    .sort_values(by = 1) # order ascending by the most popular actions
)

,0,1
action_drive_electric_vehicle,99.033816,0.966184
action_meat_free_monday,98.695652,1.304348
action_use_public_transport,93.913043,6.086957
action_no_red_meat,83.140097,16.859903
action_vegan,71.642512,28.357488
action_vegetarian,67.922705,32.077295
action_buying_secondhand,66.763285,33.236715
action_composting,62.077295,37.922705
action_using_renewable_energy,59.661836,40.338164


Values are all either 0 or 1, as specified. We know from the previous `info()` call that there are no NA values in this frame.

We see that the `action_drive_electric_vehicle` option is the least popular option with fewer than 1% of users signing up to the action. I imagine this is because of the investment required.

`action_using_renewable_energy` is the most popular action with over 40% of users signing up. This is consistent with the discussion in the white paper around the renewables action being a popular and relatively straightforward first step:

> ...no climate action is as easy or has as large an impact as switching to a renewable energy supplier.

I'm a little surprised to see `action_use_public_transport` so far down the list at approx. 6% of users. Two hypotheses here:
* Most of the users are based in big cities and so use public transport regularly (or cycle). So the action is perhaps not so relevant to them
* Most of the users are highly dependent on their cars and so are unable to make this commitment.

Perhaps some understanding of the climate conscious behaviours which users already engage in when they sign up would be useful in distinguishing between these hypotheses. As well as their location of course.


## badges

We'll repeat the steps that we used for `actions` for the `badges` fields.

These columns represent the users' values - the things they care about when making purchasing decisions.

In [14]:
# this could probably be more pandasy, but here a list comprehension will accomplish this just fine
badge_names = [n for n in user_data_raw.columns if n.startswith('badge')]
badges = user_data_raw[badge_names]
badges

,badge_reduce_waste,badge_coop,badge_socent,badge_carb_con,badge_vegan,badge_inv,badge_carb_neutral,badge_sus_sourced,badge_cruel_free,badge_fair_trade,badge_org,badge_liv_wage,badge_sup_charities
0,1,0,0,1,1,0,1,0,1,0,0,0,0
1,0,0,0,0,1,0,0,1,1,0,1,0,0
2,1,0,1,1,1,0,1,1,1,1,1,1,0
3,1,1,1,1,1,0,1,1,1,1,1,1,1
4,1,0,0,1,0,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,1,0,0,1,1,0,1,1,1,0,0,1,0
2066,1,0,0,1,0,0,1,1,1,0,0,0,1
2067,0,0,1,0,0,0,0,1,0,0,0,1,1
2068,1,0,1,0,0,0,1,1,0,1,1,0,0


We can summarise the content of the badges selections by applying a `value_count()` method to all the columns.

This will tell us the percentage of our sample who have choosen a specific badge. It will also indicate if we have any anomalous values (like a `2`) to be concerned about.

In [15]:
100 * (
    badges
    .apply(lambda x : x.value_counts(dropna = False, normalize = True))
    .fillna(0.)
    .T
    .sort_values(by = 1) # order ascending by the most popular actions
)

,0,1
badge_inv,100.000000,0.000000
badge_coop,65.942029,34.057971
badge_vegan,56.183575,43.816425
badge_sup_charities,54.927536,45.072464
badge_socent,54.830918,45.169082
badge_org,53.961353,46.038647
badge_liv_wage,34.734300,65.265700
badge_cruel_free,28.695652,71.304348
badge_fair_trade,27.294686,72.705314
badge_carb_neutral,19.855072,80.144928


Values are all either 0 or 1, as specified. We know from the previous `info()` call that there are no NA values in this frame.

Here we see an overview of the values of this sample of people who have signed up to use the Cogo app.

Perhaps unsurprisingly, we observe that the environmental values appear to be dominant: `badge_sus_sourced` (sustainably sourced), `badge_reduce_waste` (reduces waste), `badge_carb_con` (carbon conscious) and `badge_carb_neutral` (carbon neutral) being the four most popular.

Social causes such as `badge_coop` (cooperative) and `badge_sup_charities` (supports charities) appear to be less popular. I imagine this is consistent with the environmentally conscious sample of users that sign up to this app.

Exactly zero users have indicated that they care about `badge_inv`. I wonder if this is an error as roughly half of the sample are at an age where they probably care about their pension (ie. over 30). At any rate, this column isn't very informative, so I'll drop it from `user_data` prior to doing any clustering analysis.  

`badge_vegan` is a bit of an outlier. Plant-based diets are very much an environmental issue, but I wonder if a vegan diet is a bit of a stretch for most of these users. At any rate, in 2019 the Vegan Society estimated that the rate of veganism in the general UK population was 1.21% ([here](https://www.vegansociety.com/news/media/statistics/worldwide)), so the rate of veganism in our sample is orders of magnitude higher than in the general population. Perhaps this is an indicator of the impact of our self-selected sample?

## Output data

In the end we only cleaned up the `age` column and everything else looked ok (disregarding `badge_inv` for now).

I'm going to save the file as a pickled object because csvs don't handle the `Interval` objects I'm using to represent the ages very gracefully.

In [16]:
user_data.to_pickle(USER_DATA_PATH)